In [44]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
import re
from datetime import datetime
from decimal import Decimal, InvalidOperation

pd.options.display.float_format = '{:,.14f}'.format

In [8]:
load_dotenv() #add external key request for user access

slug_collection = 'boredape-baseclub'
initial_token = 'LWV2ZW50X3RpbWVzdGFtcD0yMDI1LTAyLTExKzEyJTNBMTQlM0EzNi40NzQ4ODkmLWV2ZW50X3R5cGU9Y29sbGVjdGlvbl9vZmZlciYtcGs9MzE5Nzc0MTM2Nzk='
 #first pagination 
event_type = 'order'
limit_request = 50

url = f'https://api.opensea.io/api/v2/events/collection/{slug_collection}?event_type={event_type}&limit={limit_request}'
api_key = os.getenv('api_key')

headers = {
  "accept": "application/json",
    "x-api-key": api_key  
}

In [9]:
def main_request(url, pagination_token, headers): 
    try:
        response = requests.get(url + f'&next={pagination_token}', headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None

In [ ]:
def processar_lotes(batch_size=2, delay=10,max_requests=50):
    base_dir = "datasource"
    orders_dir = os.path.join(base_dir, "orders")

    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        print(f"Pasta '{base_dir}' criada.")

    if not os.path.exists(orders_dir):
        os.makedirs(orders_dir)
        print(f"Pasta '{base_dir}' criada.")

    current_token = initial_token
    batch_number = 1
    total_paginas_processadas = 0

    while True:
        all_events = []
        request_count = 0

        for _ in range(batch_size):
            data = main_request(url,current_token, headers)
            if not data or 'asset_events' not in data:
                print("Fim dos dados ou erro na resposta")
                return
            
            all_events.extend(data['asset_events'])
            next_token = data.get('next', None)

            if not next_token or next_token == current_token:
                print("Não há mais páginas")
                if all_events:
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"events_batch_{batch_number}_{timestamp}.json"
                    filepath = os.path.join(orders_dir,filename)
                    with open(filepath, 'w') as f:
                        json.dump(all_events, f)
                    print(f"✅ Lote final salvo em {filename} (Eventos: {len(all_events)})")
                return  # Encerra a função

            current_token = next_token
            request_count += 1
            total_paginas_processadas += 1

            # Pausa após um número de requisições
            if request_count % max_requests == 0:
                print(f"⏸ Pausa de {delay} segundos...")
                time.sleep(delay)

        # Salva o lote em JSON
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"events_batch_{batch_number}_{timestamp}.json"
        filepath = os.path.join(orders_dir, filename)
        with open(filepath, 'w') as f:
            json.dump(all_events, f)
        print(f"✅ Lote {batch_number} salvo em {filepath} (Eventos: {len(all_events)})")

        batch_number += 1
        time.sleep(delay)  # Pausa entre lotes 

processar_lotes(batch_size=2, delay=10, max_requests=10)



In [2]:
pasta_json = 'datasource/orders'

dataframes = []

#making validation to collect only json files and adding into dataframes list 
for arquivo in os.listdir(pasta_json):
    if arquivo.endswith('.json'):
        caminho_completo = os.path.join(pasta_json, arquivo)

        df = pd.read_json(caminho_completo)
        dataframes.append(df)

df_orders_raw = pd.concat(dataframes, ignore_index= True)

#Creating dataframes through dictionaries series
df_asset = df_orders_raw['asset'].apply(pd.Series)
df_payment = df_orders_raw['payment'].apply(pd.Series)
df_criteria = df_orders_raw['criteria'].apply(pd.Series)
df_criteria_collection = df_criteria['collection'].apply(pd.Series)
df_criteria_contract = df_criteria['contract'].apply(pd.Series)

In [50]:

#Adding datas from dataframes above into main dataframe
df_orders = pd.concat([
    df_orders_raw.drop(columns=['asset','payment','criteria']),
    df_asset.add_prefix('asset_'),
    df_payment.add_prefix('payment_'),
    df_criteria.add_prefix('criteria_'),
    df_criteria_collection.add_prefix('criteria-collection_'),
    df_criteria_contract.add_prefix('criteria-collection_')
],axis=1)


df_orders['start_date'] = pd.to_datetime(df_orders['start_date'],unit='s')
df_orders['start_date'] = df_orders['start_date'].dt.strftime('%d-%m-%Y')
df_orders['expiration_date'] = pd.to_datetime(df_orders['expiration_date'],unit='s')
df_orders['expiration_date'] = df_orders['expiration_date'].dt.strftime('%d-%m-%Y')


In [51]:
#df_teste = df_orders[['start_date','asset_name','quantity','payment_quantity','payment_decimals','payment_symbol']][df_orders['asset_name']=='BABC #9422'].copy()
df_orders['payment_quantity_characters'] = df_orders['payment_quantity'].astype(str).str.len()
df_orders['payment_qtd'] = df_orders['payment_quantity'].astype(str).str[:12]
df_orders['payment_qtd'] = pd.to_numeric(df_orders['payment_qtd'], errors='coerce')


def calcular_payment_value(row):
    qtd_value = row['payment_quantity_characters'] - row['payment_decimals']
    if row['payment_quantity_characters'] > row['payment_decimals']:
        return int(str(row['payment_qtd'])[:qtd_value])
    elif row['payment_quantity_characters'] < row['payment_decimals']:
        return float(row['payment_qtd']/10**12)

df_orders['payment_value'] = df_orders.apply(calcular_payment_value,axis=1)

display(df_orders)





,event_type,order_hash,order_type,chain,protocol_address,start_date,expiration_date,quantity,maker,taker,...,criteria_contract,criteria_trait,criteria_encoded_token_ids,criteria-collection_0,criteria-collection_slug,criteria-collection_0,criteria-collection_address,payment_quantity_characters,payment_qtd,payment_value
0,order,0x4ccc50ea9accb57918af6c26c6bdccfa9776bd56ba7d...,listing,base,0x0000000000000068f116a894984e2db1123eb395,10-01-2025,10-01-2025,1,0x8dc817bb2b1dd2cbcda15de6b579833c86b45a44,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,530000000000,0.53000000000000
1,order,0x1ef1cc7f4c99bb317149fb4e413c09327144943ecf50...,listing,base,0x0000000000000068f116a894984e2db1123eb395,10-01-2025,10-01-2025,1,0x8dc817bb2b1dd2cbcda15de6b579833c86b45a44,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,530000000000,0.53000000000000
2,order,0xf5e20f948309c224a71017d5c622d914f07f09052380...,collection_offer,base,0x0000000000000068f116a894984e2db1123eb395,10-01-2025,10-01-2025,16,0xd4f4f6a1f386e041984da38e7be351903c767c2d,,...,{'address': '0x0be0bb1097f4465b17b1ac3b22b5427...,None,NaN,NaN,boredape-baseclub,NaN,0x0be0bb1097f4465b17b1ac3b22b5427d9cbf6d2c,17,176000000000,0.17600000000000
3,order,0xca454172e6c009c8d52c06b1f8e69da5e77f554f37db...,collection_offer,base,0x0000000000000068f116a894984e2db1123eb395,10-01-2025,10-01-2025,12,0xd4f4f6a1f386e041984da38e7be351903c767c2d,,...,{'address': '0x0be0bb1097f4465b17b1ac3b22b5427...,None,NaN,NaN,boredape-baseclub,NaN,0x0be0bb1097f4465b17b1ac3b22b5427d9cbf6d2c,16,600000000000,0.60000000000000
4,order,0xd8e06d82a5a9be9657f268c9a314a3f4d517cbb07a16...,collection_offer,base,0x0000000000000068f116a894984e2db1123eb395,10-01-2025,10-01-2025,20,0xd4f4f6a1f386e041984da38e7be351903c767c2d,,...,{'address': '0x0be0bb1097f4465b17b1ac3b22b5427...,None,NaN,NaN,boredape-baseclub,NaN,0x0be0bb1097f4465b17b1ac3b22b5427d9cbf6d2c,16,600000000000,0.60000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56695,order,0x3c86a2bf22ca41c779633b55161d20ce352781b04f7b...,listing,base,0x0000000000000068f116a894984e2db1123eb395,07-02-2025,28-02-2025,1,0xa7d99d713f8c8c3f324577572e9ae0c0bd501dc6,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,599999999999,0.59999999999900
56696,order,0x8426342460ab7d746b2d4034d164ea3d4a3920d5d197...,collection_offer,base,0x0000000000000068f116a894984e2db1123eb395,07-02-2025,07-02-2025,16,0xd4f4f6a1f386e041984da38e7be351903c767c2d,,...,{'address': '0x0be0bb1097f4465b17b1ac3b22b5427...,None,NaN,NaN,boredape-baseclub,NaN,0x0be0bb1097f4465b17b1ac3b22b5427d9cbf6d2c,17,192000000000,0.19200000000000
56697,order,0x882b0247489f5bdf0871772970a3cac305270f1ecd3b...,collection_offer,base,0x0000000000000068f116a894984e2db1123eb395,07-02-2025,07-02-2025,12,0xd4f4f6a1f386e041984da38e7be351903c767c2d,,...,{'address': '0x0be0bb1097f4465b17b1ac3b22b5427...,None,NaN,NaN,boredape-baseclub,NaN,0x0be0bb1097f4465b17b1ac3b22b5427d9cbf6d2c,16,600000000000,0.60000000000000
56698,order,0x838767166e759d65c8878ebc64ecc76a2394cb320507...,collection_offer,base,0x0000000000000068f116a894984e2db1123eb395,07-02-2025,07-02-2025,20,0xd4f4f6a1f386e041984da38e7be351903c767c2d,,...,{'address': '0x0be0bb1097f4465b17b1ac3b22b5427...,None,NaN,NaN,boredape-baseclub,NaN,0x0be0bb1097f4465b17b1ac3b22b5427d9cbf6d2c,16,600000000000,0.60000000000000
